<a href="https://colab.research.google.com/github/linyuehzzz/hedetniemi_distance/blob/master/floyd_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Floyd–Warshall Algorithm**
This code is used to implement the Floyd–Warshall Algorithm.  
Yue Lin (lin.3326 at osu.edu)  
Created: 5/30/2020

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#### **Install packages** 

In [ ]:
!pip install timeout-decorator

#### **Generate graph data** 

#####Data from the original article

In [ ]:
## [node i, node j, distance between node i and j]
## using data from example 1: San Francisco Bay Area Graph of Time-Distances (in minutes)
data = [[1, 2, 30], [1, 4, 30], [1, 9, 40],
        [2, 3, 25], [2, 4, 40], [3, 4, 50],
        [4, 5, 30], [4, 6, 20], [5, 7, 25],
        [6, 7, 20], [6, 9, 20], [7, 8, 25],
        [8, 9, 20]]
nodes = 9

##### Create random graph

In [ ]:
%cd '/content/gdrive/My Drive/Colab Notebooks/hedetniemi_matrix_sum'
import networkx as nx
import random

## Number of nodes (100/1,000/10,000/100,000/1,000,000)
nodes = 10000
print('Nodes: ', nodes)
## Total degree
degree = 3
print('Degree: ', degree)

G = nx.random_regular_graph(degree,nodes)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.uniform(1,100)
nx.draw(G)
nx.write_weighted_edgelist(G, 'graph_n' + str(nodes) + '_d' + str(degree) + '.txt')

##### Read random graph

In [ ]:
%cd '/content/gdrive/My Drive/Colab Notebooks/hedetniemi_matrix_sum'

## Number of nodes (100/1,000/10,000/100,000/1,000,000)
nodes = 100
print('Nodes: ', nodes)
## Total degree
degree = 3
print('Degree: ', degree)

data = []
with open('graph_n' + str(nodes) + '_d' + str(degree) + '.txt', 'r') as f:
  lines = f.read().splitlines()
  for line in lines:
    l = line.split()
    item = [int(l[0]), int(l[1]), float(l[2])]
    data.append(item)

print(data[0])

/content/gdrive/My Drive/Colab Notebooks/hedetniemi_matrix_sum
Nodes:  100
Degree:  3
[77, 86, 89.39726376738572]


#### **Implementation 1: list** 

##### Construct distance matrix

In [35]:
from timeit import default_timer
import timeout_decorator

@timeout_decorator.timeout(100)
def distance_matrix(graph, n):
  ## calculate distance matrix
  INF = float('inf')
  dist_mtx = [[INF] * n for i in range(n)]
  for g in graph:
    i = g[0] - 1
    j = g[1] - 1
    d = g[2]
    dist_mtx[i][j] = d
    dist_mtx[j][i] = d

  ## set diagonal to 0
  for i in range(n):
    dist_mtx[i][i] = 0.0
 
  return dist_mtx


## print time costs
start = default_timer()
dist_mtx = distance_matrix(data, nodes)
stop = default_timer()
print('Time: ', stop - start)

## print shortest path matrix
with open('dist_mtx_list.txt', 'w') as fw:
  fw.write('\n'.join(['\t'.join([str(round(cell,2)) for cell in row]) for row in dist_mtx]))

Time:  0.0004924960003336309


##### Calculate Floyd–Warshall distance

In [36]:
from timeit import default_timer
import timeout_decorator

@timeout_decorator.timeout(10)
def floyd_distance(matrix, n):
  for k in range(n):
    for i in range(n):
      for j in range(n):
        if matrix[i][j] > matrix[i][k] + matrix[k][j]:
          matrix[i][j] = matrix[i][k] + matrix[k][j]
  
  return matrix


## print time costs
try:
  start = default_timer()
  mtx_a_t = floyd_distance(dist_mtx, nodes)
  stop = default_timer()
  print('Time: ', stop - start)
except:
  print('Time: inf')

## print shortest path matrix
with open('floyd_mtx_list.txt', 'w') as fw:
  fw.write('\n'.join(['\t'.join([str(round(cell,2)) for cell in row]) for row in mtx_a_t]))

Time:  0.12761181000041688


#### **Implementation 2: numpy** 

##### Construct distance matrix

In [39]:
from timeit import default_timer
import numpy as np
import timeout_decorator

@timeout_decorator.timeout(100)
def distance_matrix(graph, n):
  ## calculate distance matrix
  dist_mtx = np.full((n,n), np.inf)
  for g in graph:
    i = int(g[0]) - 1
    j = int(g[1]) - 1
    d = g[2]
    dist_mtx[i,j] = d
    dist_mtx[j,i] = d

  ## set diagonal to 0
  np.fill_diagonal(dist_mtx, 0)
 
  return dist_mtx


## print time costs
start = default_timer()
dist_mtx = distance_matrix(np.array(data), nodes)
stop = default_timer()
print('Time: ', stop - start)

## print distance matrix
# print("Distance matrix: ")
# for line in dist_mtx:
#   print(line)

Time:  0.0006069540004318696


##### Calculate Floyd–Warshall distance

In [40]:
from timeit import default_timer
import numpy as np
import timeout_decorator

@timeout_decorator.timeout(100)
def floyd_distance(matrix, n):
  for k in range(n):
    for i in range(n):
      for j in range(n):
        if matrix[i,j] > matrix[i,k] + matrix[k,j]:
          matrix[i,j] = matrix[i,k] + matrix[k,j]
  
  return matrix


## print time costs
start = default_timer()
mtx_a_t = floyd_distance(dist_mtx, nodes)
stop = default_timer()
print('Time: ', stop - start)

Time:  0.7044710130003295


#### **Implementation 3: numba (njit)** 

##### Construct distance matrix

In [41]:
from timeit import default_timer
import numpy as np
import numba
import timeout_decorator

@timeout_decorator.timeout(10)
@numba.njit
def distance_matrix(graph, n):
  ## calculate distance matrix
  dist_mtx = np.full((n,n), np.inf)
  for g in numba.prange(graph.shape[0]):
    i = int(graph[g,0]) - 1
    j = int(graph[g,1]) - 1
    d = graph[g,2]
    dist_mtx[i,j] = d
    dist_mtx[j,i] = d

  ## set diagonal to 0
  np.fill_diagonal(dist_mtx, 0)
 
  return dist_mtx


## print time costs
try:
  start = default_timer()
  dist_mtx = distance_matrix(np.array(data), nodes)
  stop = default_timer()
  print('Time: ', stop - start)
except:
  print('Time: inf')

## print shortest path matrix
with open('dist_mtx_nb.txt', 'w') as fw:
  fw.write('\n'.join(['\t'.join([str(round(cell,2)) for cell in row]) for row in dist_mtx]))

Time:  0.24978987099984806


##### Calculate Floyd–Warshall distance

In [42]:
from timeit import default_timer
import numpy as np
import numba
import timeout_decorator

@numba.njit
def all_pair_floyd(matrix, k, i, n):
  for j in numba.prange(n):
    if matrix[i,j] > matrix[i,k] + matrix[k,j]:
      matrix[i,j] = matrix[i,k] + matrix[k,j]


@timeout_decorator.timeout(100)
def floyd_distance(matrix, n):
  for k in range(n):
    for i in range(n):
      for j in range(n):
        if matrix[i,j] > matrix[i,k] + matrix[k,j]:
          matrix[i,j] = matrix[i,k] + matrix[k,j]
      # all_pair_floyd(matrix, k, i, n)
  
  return matrix


## print time costs
try:
  start = default_timer()
  mtx_a_t = floyd_distance(dist_mtx, nodes)
  stop = default_timer()
  print('Time: ', stop - start)
except:
  print('Time: inf')
  raise

## print shortest path matrix
with open('floyd_mtx_nb.txt', 'w') as fw:
  fw.write('\n'.join(['\t'.join([str(round(cell,2)) for cell in row]) for row in mtx_a_t]))

Time:  0.6899059490006039


In [31]:
!diff dist_mtx_list.txt dist_mtx_nb.txt